In [1]:
using POMDPs
using Random 
using Parameters
using StaticArrays
using Distributions
using MCTS
using POMDPSimulators

In [2]:
const Vec2 = SVector{2, Float64}
const Vec3 = SVector{3, Float64}

struct mystate
    uavPose::Vec3
    uavHeading::Float64 # radius
    targetPose::Vec2
end

struct myaction
    xy_speed::Float64 # m/s
    z_speed::Float64
    angle::Float64 # radius
end

@with_kw mutable struct UAVMDP <: MDP{mystate, myaction}

    target_velocity::Vec2 = SVector(0.5,0.5)
    target_std::Float64 = 0.1

    r_outScene::Float64 = -70
    r_action::Float64 = -10
    r_reach::Float64 = 100

    discount::Float64 = 0.95
    
    boundary::Int = 100
    landing_r::Int = 1

end

UAVMDP

In [3]:
function POMDPs.isterminal(p::UAVMDP,s::mystate,a::myaction)
    condition1 = (sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2) < p.landing_r)
    condition2 = s.uavPose[1] > p.boundary || s.uavPose[2]>p.boundary || s.uavPose[3]>p.boundary
    condition3 = s.uavPose[1] < 0 || s.uavPose[2] < 0 || s.uavPose[3] < 0
    condition = condition1 || condition2 || condition3
    return condition
end

In [4]:
function POMDPs.generate_s(p::UAVMDP, s::mystate, a::myaction)
    # calculate target state
    target_dt_distance = p.target_velocity
    sensor_noise = rand(Normal(0, p.target_std), 2)
    curr_targ = s.targetPose + target_dt_distance + sensor_noise # next_target_pos(p, s.targetPose)
    # calculate UAV state
    curr_angle = s.uavHeading + a.angle
    xy_dt_distance = a.xy_speed*SVector(cos(curr_angle), sin(curr_angle)) # careful
    z_dt_distance = a.z_speed
    xyz_dt_distance = SVector(xy_dt_distance[1], xy_dt_distance[2], z_dt_distance)
    curr_pos = s.uavPose + xyz_dt_distance
    return mystate(curr_pos, curr_angle, curr_targ)
end

In [5]:
function myreward(p::UAVMDP, s::mystate, a::myaction)
    
    distance_to_target = sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2)
    reward = p.r_action + 1/(distance_to_target+1)
    
    #out scene punish 
    if s.uavPose[1]>p.boundary || s.uavPose[2]>p.boundary
        reward = reward + p.r_outScene
    end
    
    if distance_to_target < p.landing_r
        reward = reward + p.r_reach
    end
    return reward
end

myreward (generic function with 1 method)

In [6]:
function POMDPs.actions(p::UAVMDP)
    action_space = [myaction(i, j, k) for i=-5.0:5.0,j=-5.0:5.0,k=-pi:pi]
    return action_space
end

In [7]:
function POMDPs.generate_sr(p::UAVMDP, s::mystate, a::myaction, rng::AbstractRNG)
    sp = generate_s(p, s, a)
    r = myreward(p, s, a)
    return sp, r
end

In [19]:
POMDPs.initialstate_distribution(p::UAVMDP) = mystate(SVector(rand(1:100),rand(1:100),rand(1:100)),0,SVector(0,0))
POMDPs.discount(p::UAVMDP) = p.discount
solver = MCTSSolver(n_iterations=50, depth=20, exploration_constant=5.0)
UAV = UAVMDP()

UAVMDP
  target_velocity: SArray{Tuple{2},Float64,1,2}
  target_std: Float64 0.1
  r_outScene: Float64 -70.0
  r_action: Float64 -10.0
  r_reach: Float64 100.0
  discount: Float64 0.95
  boundary: Int64 100
  landing_r: Int64 1


In [20]:
policy = solve(solver, UAV)

MCTSPlanner{UAVMDP,mystate,myaction,MCTS.SolvedRolloutEstimator{POMDPPolicies.RandomPolicy{MersenneTwister,UAVMDP,BeliefUpdaters.NothingUpdater},MersenneTwister},MersenneTwister}(MCTSSolver(50, 20, 5.0, MersenneTwister(UInt32[0x384992b1, 0x4d4186f6, 0x65d3ca8f, 0x0f870daa], Random.DSFMT.DSFMT_state(Int32[1666487707, 1073230605, -659266473, 1073148275, -1244187148, 1072794303, -77635571, 1072847558, 172358740, 1073704220  …  812870699, 1073041398, 1060184256, 1073740245, -1253345191, -727061896, 1196420479, -1690985851, 382, 0]), [1.69662, 1.64343, 1.76243, 1.85251, 1.24353, 1.47309, 1.74671, 1.52653, 1.81725, 1.07697  …  1.24999, 1.33611, 1.53986, 1.45684, 1.42636, 1.84451, 1.74536, 1.72495, 1.33202, 1.99849], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000000000

In [22]:
uav_poses = [] # Vector of sequential uav poses
target_poses = []

0-element Array{Any,1}

In [23]:
i=0
terminate = false
reward_sum = 0

current_state = mystate(SVector(rand(1:100),rand(1:100),rand(1:100)),0,SVector(0,0)) ##initialize state

while i<500 && !terminate
    current_action = action(policy,current_state)
    current_reward = myreward(UAV, current_state, current_action)
    current_state = generate_s(UAV, current_state, current_action)
    push!(uav_poses, current_state.uavPose)
    push!(target_poses, current_state.targetPose)
    terminate = isterminal(UAV,current_state,current_action)
    reward_sum = reward_sum + current_reward
    i=i+1
    @show current_state
end

current_state = mystate([20.0, 3.0, 76.0], -3.141592653589793, [0.475221, 0.536542])
current_state = mystate([21.0, 3.0, 75.0], -6.283185307179586, [0.882317, 1.04368])
current_state = mystate([20.0, 3.0, 74.0], -9.42477796076938, [1.49502, 1.83431])
current_state = mystate([21.0, 3.0, 73.0], -12.566370614359172, [1.73799, 2.2896])
current_state = mystate([20.0, 3.0, 72.0], -15.707963267948966, [2.1499, 2.69333])
current_state = mystate([21.0, 3.0, 71.0], -18.84955592153876, [2.7326, 3.18587])
current_state = mystate([20.0, 3.0, 70.0], -21.991148575128552, [3.43275, 3.62933])
current_state = mystate([21.0, 3.0, 69.0], -25.132741228718345, [4.10788, 4.13088])
current_state = mystate([20.0, 3.0, 68.0], -28.274333882308138, [4.77178, 4.66245])
current_state = mystate([21.0, 3.0, 67.0], -31.41592653589793, [5.19372, 5.02242])
current_state = mystate([20.0, 3.0, 66.0], -34.55751918948772, [5.83229, 5.43447])
current_state = mystate([21.0, 3.0, 65.0], -37.69911184307752, [6.31748, 5.73688])


In [13]:
using Pkg
Pkg.add("DelimitedFiles")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [17]:
using DelimitedFiles
writedlm( "./uvapos.csv", uav_poses)
writedlm( "./targetpos.csv", target_poses)

31